<a href="https://colab.research.google.com/github/physicaone/loss_IG/blob/master/%5B210805%5Dcheck_Bias_convergence.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
try:
    from google.colab import drive
    drive.mount('/content/drive')
    base='drive/MyDrive'
except:
    base='Google Drive'

Mounted at /content/drive


In [ ]:
import numpy as np
import torch
import torchvision.datasets
import torchvision.models
import torchvision.transforms
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms
from torchvision.utils import make_grid
import torch.utils.data
import torch.nn as nn
from datetime import datetime
from tqdm import tqdm, tnrange
import warnings
warnings.filterwarnings("ignore")

import random
import pickle as pkl
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
%matplotlib inline
# np.seterr(divide='ignore', invalid='ignore')
import itertools
import re
from time import sleep
from scipy.stats import entropy


In [ ]:
class RBM(nn.Module):

    def __init__(self, n_vis, n_hid, k, use_cuda):
        """Create a RBM."""
        super(RBM, self).__init__()
        
        if use_cuda==True:
            self.v = nn.Parameter(torch.ones(1, n_vis).cuda())
            self.h = nn.Parameter(torch.zeros(1, n_hid).cuda())
            self.W = nn.Parameter(torch.randn(n_hid, n_vis).cuda())
            self.k = k
        else:
            self.v = nn.Parameter(torch.ones(1, n_vis))
            self.h = nn.Parameter(torch.zeros(1, n_hid))
            self.W = nn.Parameter(torch.randn(n_hid, n_vis))
            self.k = k     

    def visible_to_hidden(self, v, beta):
        return torch.sigmoid(F.linear(v, self.W, self.h)*beta)

    def hidden_to_visible(self, h, beta):
        return torch.sigmoid(F.linear(h, self.W.t(), self.v)*beta)

    def free_energy(self, v):
        v_term = torch.matmul(v, self.v.t())
        w_x_h = F.linear(v, self.W, self.h)
        h_term = torch.sum(F.softplus(w_x_h), dim=1)
        return torch.mean(-h_term - v_term)
    
    def energy(self, v):
        v=v.bernoulli()
        h=torch.sigmoid(F.linear(v, self.W, self.h))
        h=h.bernoulli()
        return -torch.matmul(v, self.v.t())-torch.matmul(torch.matmul(v, self.W.t()),h.t())-torch.matmul(h, self.h.t())
    
    def energy2(self, v, h):
        v=v.bernoulli()
        h=h.bernoulli()
        return -torch.matmul(v, self.v.t())-torch.matmul(torch.matmul(v, self.W.t()),h.t())-torch.matmul(h, self.h.t())

    def forward(self, v):
        h = self.visible_to_hidden(v,1)
        h = h.bernoulli()
        for _ in range(self.k):
            v_gibb = self.hidden_to_visible(h,1).to(device)
            v_gibb = v_gibb.bernoulli()
            h = self.visible_to_hidden(v_gibb, 1).to(device)
            h = h.bernoulli()
        return v, v_gibb

In [ ]:
CUDA = torch.cuda.is_available()
CUDA_DEVICE = 0

if CUDA:
    device='cuda'
else:
    device='cpu'
torch.cuda.is_available(); CUDA

True

In [ ]:
# 아래 두 함수는 PT와는 아무 상관 없습니다. 10진법을 2진법으로 바꾸고 리스트 형태로 변형하는 함수입니다. 예)15->[1,1,1,1]
# 그리고 그 다음은 역함수입니다.
def decimal_to_binary(integer, n_hid):
    string=bin(int(integer))[2:]
    list0=[float(d) for d in string]
    while len(list0)<n_hid:
        list0=[0.]+list0
    return torch.tensor([list0])

def binary_to_decimal(list0):
    value=0
    list0=list0.tolist()
    for i in range(len(list0)):
        value+=list0[-i-1]*2**(i)
    return int(value)

def flatten_list(list0):
    flattened = [val for sublist in list0 for val in sublist]
    return flattened

# Energy_GPU returns an array of values
def Energy_GPU2(model0_dict, v_list0, h_list0):
    n_split=1

    values=[]
    vs=[]
    hs=[]
    for i in range(n_split):
        v_list=torch.stack(list(v_list0[i*int(len(v_list0)/n_split):(i+1)*int(len(v_list0)/n_split)]))
        h_list=torch.stack(list(h_list0[i*int(len(h_list0)/n_split):(i+1)*int(len(h_list0)/n_split)]))
        v_list=v_list.detach().to(device).view(len(v_list), n_vis)
        h_list=h_list.detach().to(device).view(len(h_list), n_hid)
        vs.append(v_list)
        hs.append(h_list)
    for i in range(n_split):  
        a=model0_dict['v'].detach().to(device).view(n_vis)
        b=model0_dict['h'].detach().to(device).view(n_hid)
        W=model0_dict['W'].detach().to(device).view(n_hid, n_vis)
        e=-torch.matmul(vs[i].float(), a)-torch.diagonal(torch.matmul(torch.matmul(vs[i].float(), W.t()), hs[i].float().t()))-torch.matmul(hs[i].float(), b)
        values.append(e.cpu().detach().numpy())

    return np.array(flatten_list(values))

def train_and_get_data(n_hid, model, lr, train_loader):
    if model==0:
        rbm = RBM(n_vis, n_hid, k, use_cuda=CUDA)
    else:
        rbm=model
    train_loss_list=[]
#     train_op = optim.Adam(rbm.parameters(), lr)
    train_op = optim.SGD(rbm.parameters(), lr=lr, momentum=0.9)
    rbm.train()
    for epoch in tnrange(n_epochs):
        train_loss_epoch = []
        for _, (data) in enumerate(train_loader):
            data=data.to(device)
            v, v_gibbs = rbm(data.view(-1, n_vis))
            train_loss = rbm.free_energy(v) - rbm.free_energy(v_gibbs)
            train_loss_epoch.append(train_loss.item())
            train_op.zero_grad()
            train_loss.backward()
            train_op.step()
        val=np.mean(train_loss_epoch)
        train_loss_list.append(val)
#         print(epoch, val)
#         if epoch>1 and train_loss_list[-1]*train_loss_list[-2]<0:
#             train_op = optim.SGD(rbm.parameters(), lr=0.001*lr, momentum=0.9)

        #     break
    # with open('drive/MyDrive/loss_IG/3*3/loss/2021-05-02_loss_n_hid={n_hid}_{por}_T={T}.pkl'.format(n_hid=n_hid, por=por, T=T), 'wb') as f:
    #     pkl.dump(train_loss_list, f)
    # torch.save(rbm.state_dict(), 'drive/MyDrive/loss_IG/3*3/state_dict/2021-05-02_n_hid={n_hid}_{por}_T={T}'.format(n_hid=n_hid, por=por, T=T))
    rbm=rbm.cpu()
    return rbm.state_dict()

def CM_model(models):
    new_v_bias=0; new_h_bias=0; new_Weight=0
    for i in range(10):
        new_v_bias+=models[str(i)]['v']/10
        new_h_bias+=models[str(i)]['h']/10
        new_Weight+=models[str(i)]['W']/10
    return {'v':new_v_bias, 'h':new_h_bias, 'W':new_Weight}  

def Rearrange_parameters(model0):
    W=model0['W']
    Wmean={}
    for i in range(len(W)):
        Wmean[torch.mean(W[i])]=i
    sorted_mean=sorted(Wmean.keys())
    sorted_index=[Wmean[j] for j in sorted_mean]
    sorted_weight=torch.stack([W[k] for k in sorted_index])
    sorted_b=torch.tensor([[model0['h'][0][k] for k in sorted_index]])
    
    return {'W':sorted_weight, 'v':model0['v'], 'h':sorted_b}

def Ising_energy(v_list):
    L = 3
    E_list=[]
    for n in range(len(v_list)):
        v=v_list[n]
        E = 0
        for i in range(L):
            for j in range(L):
                s = v[i,j]
                neigh = v[(i+1)%L, j] + v[i,(j+1)%L] + v[(i-1)%L,j] + v[i,(j-1)%L] 
                E += -neigh * s
        E_list.append(E/2)
    return np.array(E_list)
    
def FE_V(model0, v0):
#     v0=torch.tensor(v0)
    a=model0['v'].detach()
    b=model0['h'].detach()
    W=model0['W'].detach()
    values=[]
    for i in range(len(v0)):
        v_term = torch.matmul(v0[i].detach(), a.t()).detach()
        w_x_h = F.linear(v0[i].detach(), W, b).detach()
        h_term = torch.sum(F.softplus(w_x_h), dim=1).detach()
        values.append((-h_term - v_term).detach().numpy())
    return np.squeeze(values)

from torch.utils.data import Dataset

class CustomDataset(Dataset): 
    def __init__(self, dataset):
        data_x = dataset
        self.x_data = data_x
#         self.y_data = data_y

    # 총 데이터의 개수를 리턴
    def __len__(self): 
        return len(self.x_data)
    # 인덱스를 입력받아 그에 맵핑되는 입출력 데이터를 파이토치의 Tensor 형태로 리턴
    def __getitem__(self, idx): 
        x = torch.FloatTensor(self.x_data[idx])
#         y = torch.FloatTensor([self.y_data[idx]])
        return x

In [ ]:
T=2.3
n_hid=12
n_vis=9
k=5
n_epochs=500
batch_size=512
lr=0.03
# T='1.47'

fullconfigs=pd.read_pickle('{base}/loss_IG/3*3/3*3_full_T={T}.pkl'.format(base=base, T=T))
def data_to_loader(fullconfigs):
    fulldata=CustomDataset(fullconfigs)
    full_dataset = fulldata
    full_loader = torch.utils.data.DataLoader(full_dataset, batch_size)
    return full_loader
loader_list=[]
for i in range(10):
    loader_list.append(data_to_loader(fullconfigs[i]))
    # loader_list.append(data_to_loader(fullconfigs[int(len(fullconfigs)*i/10):int(len(fullconfigs)*(i+1)/10)]))

model_dicts_test=train_and_get_data(n_hid,0,lr,train_loader=loader_list[0])
n_split=2**8
Z=0
E=0
for j in tnrange(n_split):
    v_list=[]; h_list=[]
    for s in range(j*int(2**(n_vis+n_hid)/n_split), (j+1)*int(2**(n_vis+n_hid)/n_split)):
        full=decimal_to_binary(s, n_hid+n_vis)[0]
        v=full[:n_vis]; h=full[-n_hid:]
        v_list.append(v); h_list.append(h)
    v_list0=torch.stack(v_list)
    h_list0=torch.stack(h_list)

    E=Energy_GPU2(model_dicts_test, v_list0, h_list0)
    Z+=sum(np.exp(-E))

In [ ]:
def Ising_energy(v_list):
    L = 3
    E_list=[]
    for n in range(len(v_list)):
        v=v_list[n]
        E = 0
        for i in range(L):
            for j in range(L):
                s = v[i,j]
                neigh = v[(i+1)%L, j] + v[i,(j+1)%L] + v[(i-1)%L,j] + v[i,(j-1)%L] 
                E += -neigh * s
        E_list.append(E/2)
    return np.array(E_list)
    
def FE_V(model0, v0):
#     v0=torch.tensor(v0)
    a=model0['v'].detach()
    b=model0['h'].detach()
    W=model0['W'].detach()
    values=[]
    for i in range(len(v0)):
        v_term = torch.matmul(v0[i].detach(), a.t()).detach()
        w_x_h = F.linear(v0[i].detach(), W, b).detach()
        h_term = torch.sum(F.softplus(w_x_h), dim=1).detach()
        values.append((-h_term - v_term).detach().numpy())
    return np.squeeze(values)

v_list=[]
v_list2=[]
for s in range(2**n_vis):
    v=decimal_to_binary(s, n_vis)[0]
    v_list.append(np.reshape(v,(3,3))*2-1)
    v_list2.append(np.reshape(v,(1, 9)))
for T in [1.78]:
    bf_list=np.exp(-Ising_energy(v_list)/T)
    Pv=bf_list/sum(bf_list)


In [ ]:
v_list=[]
v_list2=[]
for s in range(2**n_vis):
    v=decimal_to_binary(s, n_vis)[0]
    v_list.append(np.reshape(v,(3,3))*2-1)
    v_list2.append(np.reshape(v,(1, 9)))
for T in [1.78]:
    bf_list=np.exp(-Ising_energy(v_list)/T)
    Pv=bf_list/sum(bf_list)


In [ ]:

-entropy(Pv)+np.log(Z)+np.dot(FE_V(model_dicts_test, v_list2),Pv)

0.13351027250116942